In [1]:
# Package installs
%pip install pandas
%pip install matplotlib
%pip install numpy
%pip install seaborn
%pip install scikit-learn
%pip install cartopy
%pip install keras
%pip install keras-tuner
# you may have to install in console using pip install tensorflow --user or pip install tensorflow --upgrade
%pip install tensorflow
%pip install basketball-refrence-scraper

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated pac

ERROR: Could not find a version that satisfies the requirement basketball-refrence-scraper (from versions: none)
ERROR: No matching distribution found for basketball-refrence-scraper


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# Modeling tools
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
import keras_tuner as kt
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import OutputType
from datetime import date

In [3]:
player_stats = client.players_season_totals(season_end_year=2025)
print(player_stats)  # List of all players' season stats

[{'slug': 'gilgesh01', 'name': 'Shai Gilgeous-Alexander', 'positions': [<Position.POINT_GUARD: 'POINT GUARD'>], 'age': 26, 'team': <Team.OKLAHOMA_CITY_THUNDER: 'OKLAHOMA CITY THUNDER'>, 'games_played': 56, 'games_started': 56, 'minutes_played': 1917, 'made_field_goals': 627, 'attempted_field_goals': 1200, 'made_three_point_field_goals': 116, 'attempted_three_point_field_goals': 319, 'made_free_throws': 447, 'attempted_free_throws': 501, 'offensive_rebounds': 49, 'defensive_rebounds': 242, 'assists': 346, 'steals': 104, 'blocks': 59, 'turnovers': 146, 'personal_fouls': 128, 'points': 1817}, {'slug': 'edwaran01', 'name': 'Anthony Edwards', 'positions': [<Position.SHOOTING_GUARD: 'SHOOTING GUARD'>], 'age': 23, 'team': <Team.MINNESOTA_TIMBERWOLVES: 'MINNESOTA TIMBERWOLVES'>, 'games_played': 57, 'games_started': 57, 'minutes_played': 2097, 'made_field_goals': 520, 'attempted_field_goals': 1192, 'made_three_point_field_goals': 236, 'attempted_three_point_field_goals': 581, 'made_free_throws'

In [4]:
df1 = pd.DataFrame(player_stats)

In [5]:
df2 = pd.DataFrame(player_stats)

# Calculate per-game averages
df2["points_per_game"] = df2["points"] / df2["games_played"]
df2["assists_per_game"] = df2["assists"] / df2["games_played"]
df2["total_rebounds"] = df2["offensive_rebounds"] + df2["defensive_rebounds"]
df2["rebounds_per_game"] = df2["total_rebounds"] / df2["games_played"]
df2["steals_per_game"] = df2["steals"] / df2["games_played"]
df2["blocks_per_game"] = df2["blocks"] / df2["games_played"]
df2["turnovers_per_game"] = df2["turnovers"] / df2["games_played"]

# Display only relevant columns
df2 = df2[["name", "points_per_game", "assists_per_game", "rebounds_per_game","blocks_per_game" , "turnovers_per_game"]]

In [6]:
df2.to_csv("average_stats.csv", index=False)

In [7]:
all_games_2025 = client.season_schedule(season_end_year=2025)
df_2025 = pd.DataFrame(all_games_2025)

csv_filename = "NBA_2025_schedule.csv"
teamlist = []
for team in df_2025['away_team']:
    add = True
    for i in teamlist:
        if team == i:
            add = False
    if(add):        
        teamlist.append(team)

df_tlist =pd.DataFrame(teamlist)
df_tlist.to_csv("ListOfTeams.csv",index = False)
df_2025.to_csv( "NBA_2025_schedule.csv", index = False)      

In [8]:
df = pd.read_csv("NBA_2025_schedule.csv")


df = df.dropna(subset=['away_team_score'])

# Save the cleaned DataFrame
df.to_csv("NBA_2025_cleaned_schedule.csv", index=False)
        
        


'''df['away_team'] = df['away_team'].str.replace("TEAM.", '', regex=True)
df['home_team'] = df['home_team'].str.replace("TEAM.", "Bruh", regex=True)
df.to_csv( "NBA_20256_schedule.csv", index = False)'''      

'df[\'away_team\'] = df[\'away_team\'].str.replace("TEAM.", \'\', regex=True)\ndf[\'home_team\'] = df[\'home_team\'].str.replace("TEAM.", "Bruh", regex=True)\ndf.to_csv( "NBA_20256_schedule.csv", index = False)'

In [9]:
df = pd.read_csv("NBA_2025_cleaned_schedule.csv")

df['away_team'] = df['away_team'].str.replace('Team.','',)
df['home_team'] = df['home_team'].str.replace('Team.','',)

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [10]:
df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
df1 = pd.read_csv("ListOfTeams.csv")
df['TEAMS'] = df1

df.to_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv", index=False)

In [11]:

df = pd.read_csv("NBA_2025_OFFICIAL_TEAM_SCHEDULE.csv")
calcdf = []

for i in range(len(df)):
    home_score = df.at[i, 'home_team_score']
    away_score = df.at[i, 'away_team_score']
    home_team = df.at[i, 'home_team']
    away_team = df.at[i, 'away_team']

    home_won = 1 if home_score > away_score else 0
    away_won = 1 if away_score > home_score else 0

    # Append home team results
    calcdf.append({
        'TEAM': home_team,
        'OPPONENT': away_team,
        'TEAM_HOME': 1,  # 1 = Home Game
        'WIN': home_won,
        'TEAM_SCORE': home_score,
        'OPPONENT_SCORE': away_score
    })

    # Append away team results
    calcdf.append({
        'TEAM': away_team,
        'OPPONENT': home_team,
        'TEAM_HOME': 0,  # 0 = Away Game
        'WIN': away_won,
        'TEAM_SCORE': away_score,
        'OPPONENT_SCORE': home_score
    })

# Convert to DataFrame
dflonger = pd.DataFrame(calcdf)

# Group and summarize matchups properly
dfsummary = dflonger.groupby(['TEAM', 'OPPONENT', 'TEAM_HOME']).agg(
    games_played=('WIN', 'size'),
    wins=('WIN', 'sum'),
    total_points_scored=('TEAM_SCORE', 'sum'),
    total_points_allowed=('OPPONENT_SCORE', 'sum'),
    avg_points_scored=('TEAM_SCORE', 'mean'),
    avg_points_allowed=('OPPONENT_SCORE', 'mean')
).reset_index()

# Save to CSV
dfsummary.to_csv("NBA_Team_Performance.csv", index=False)

# Display summary
print(dfsummary)

                    TEAM           OPPONENT  TEAM_HOME  games_played  wins  \
0          ATLANTA_HAWKS     BOSTON_CELTICS          0             2     2   
1          ATLANTA_HAWKS     BOSTON_CELTICS          1             1     0   
2          ATLANTA_HAWKS      BROOKLYN_NETS          1             1     1   
3          ATLANTA_HAWKS  CHARLOTTE_HORNETS          0             1     1   
4          ATLANTA_HAWKS  CHARLOTTE_HORNETS          1             1     1   
...                  ...                ...        ...           ...   ...   
1323  WASHINGTON_WIZARDS       PHOENIX_SUNS          1             1     0   
1324  WASHINGTON_WIZARDS   SACRAMENTO_KINGS          0             1     0   
1325  WASHINGTON_WIZARDS  SAN_ANTONIO_SPURS          0             1     0   
1326  WASHINGTON_WIZARDS  SAN_ANTONIO_SPURS          1             1     0   
1327  WASHINGTON_WIZARDS    TORONTO_RAPTORS          1             1     0   

      total_points_scored  total_points_allowed  avg_points_sco